# Logistical Regression Analysis:

### Covers data loading, preprocessing, exploratory data analysis, feature selection, model training, evaluation, result interpretation and prediction for a new input.

**Introduction**
This project aims to predict whether it will rain tomorrow in Australia based on historical weather data using a logistic regression model.

**Dataset context**
This dataset sourced from kaggle contains about 10 years of daily weather observations from numerous Australian weather stations.

RainTomorrow is the target variable to predict. It means, did it rain the next day - Yes or No?
This column is Yes if the rain for that day was 1mm or more.

https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

In [ ]:
# Import necessary libraries for data analysis and modeling

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical computations
import matplotlib.pyplot as plt # For data visualisation
import seaborn as sns # For enhanced data visualisation

from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler  # For feature standardisation
from sklearn.impute import SimpleImputer  # For handling missing values
from sklearn.linear_model import LogisticRegression  # For logistic regression model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # For model evaluation